## Imports

In [6]:
# Data Privacy Final Project
# Jordan Bourdeau, Casey Forey
import numpy as np
import os
import pandas as pd
import requests
from sklearn.linear_model import LogisticRegression
import zipfile

## Data

In [19]:
url: str = "https://jbourde2.w3.uvm.edu/data-privacy/data.zip"
file_path: str = "data/powerlifting-data.csv"


# If the .zip file doesn't already exist, download it from the Silk server.
if not os.path.exists("data.zip"):
    try:
        r = requests.get(url, allow_redirects=True)
        print("Downloading zip file from server")
        open("data.zip", "wb").write(r.content)
        print("Zip file downloaded from server")
    except Exception as e:
        print(e)
        print("Unable to download zip file from remote server")
        exit(1)

# If the data folder doesn't already exist, unzip the data zip
if not os.path.exists("data/"):
    try:
        with zipfile.ZipFile('data.zip') as zip:
            zip.extractall()
        print("Zip file extracted")
        df = pd.read_csv(file_path)
        print("Data read in")
    except Exception as e:
        print(e)
        print("No zip file to extract from")
        exit(1)

if "df" not in locals():
    print("Loading dataframe")
    df = pd.read_csv(file_path)
    # Drop unneeded columns
    print(df.columns)
    df = df.drop(["AgeClass", "BirthYearClass", "Dots", "Wilks", "Glossbrenner", "Goodlift", "ParentFederation", "MeetCountry", "MeetState", "MeetTown", "MeetName"], axis=1)
    print(df.columns)
    print("Dataframe loaded")
else:
    print("Dataframe already loaded")

Dataframe already loaded
Index(['Name', 'Sex', 'Event', 'Equipment', 'Age', 'AgeClass',
       'BirthYearClass', 'Division', 'BodyweightKg', 'WeightClassKg',
       'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Best3SquatKg',
       'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Best3BenchKg',
       'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg',
       'Best3DeadliftKg', 'TotalKg', 'Place', 'Dots', 'Wilks', 'Glossbrenner',
       'Goodlift', 'Tested', 'Country', 'State', 'Federation',
       'ParentFederation', 'Date', 'MeetCountry', 'MeetState', 'MeetTown',
       'MeetName'],
      dtype='object')
Index(['Name', 'Sex', 'Event', 'Equipment', 'Age', 'Division', 'BodyweightKg',
       'WeightClassKg', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg',
       'Best3SquatKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg',
       'Best3BenchKg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg',
       'Deadlift4Kg', 'Best3DeadliftKg', 'TotalKg', 'Place', 'Tested',
       'Count

## Algorithms 

In [8]:
"""
Differentially private mechanisms.
"""
def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def gaussian_mech_RDP_vec(vec, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def gaussian_mech_zCDP_vec(vec, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]


""" 
Machine learning functions (loss, gradient, noisy gradient descent).
"""

# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the logistic loss function.
def loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

def avg_grad(theta, X, y):
    grads = [gradient(theta, xi, yi) for xi, yi in zip(X, y)]
    return np.mean(grads, axis=0)

# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

def accuracy(theta, X_test, y_test):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

# L2 Clipping
def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

def gradient_sum(theta, X, y, b):
    gradients = [L2_clip(gradient(theta, x_i, y_i), b) for x_i, y_i in zip(X,y)]
        
    # sum query
    # L2 sensitivity is b (by clipping performed above)
    return np.sum(gradients, axis=0)

"""
Noisy gradient descent algorithms.
"""
    
# Noisy gradient descent
# Satisfies (k*epsilon + epsilon, k*delta)-differential privacy
def noisy_gradient_descent(X_train, y_train, iterations, epsilon, delta):
    theta = np.zeros(X_train.shape[1])
    b = 3

    noisy_count = laplace_mech(X_train.shape[0], 1, epsilon)

    for i in range(iterations):
        clipped_gradient_sum = gradient_sum(theta, X_train, y_train, b)
        noisy_gradient_sum = np.array(gaussian_mech_vec(clipped_gradient_sum, b, epsilon, delta))
        noisy_avg_gradient = noisy_gradient_sum / noisy_count
        theta = theta - noisy_avg_gradient

    return theta

def noisy_gradient_descent_RDP(X_train, y_train, iterations, alpha, epsilon_bar):
    theta = np.zeros(X_train.shape[1])
    b = 3
    epsilon_bar_count = 0.05 * epsilon_bar
    epsilon_bar_i = 0.95 * epsilon_bar / iterations
    
    noisy_count = gaussian_mech_RDP_vec([len(X_train)], 1, alpha, epsilon_bar_count)[0]

    for i in range(iterations):
        clipped_gradient_sum = gradient_sum(theta, X_train, y_train, b)
        noisy_gradient_sum = np.array(gaussian_mech_RDP_vec(clipped_gradient_sum, b, alpha, epsilon_bar_i))
        noisy_avg_gradient = noisy_gradient_sum / noisy_count
        theta = theta - noisy_avg_gradient

    return theta

def noisy_gradient_descent_zCDP(X_train, y_train, iterations, rho):
    theta = np.zeros(X_train.shape[1])
    b = 3
    rho_count = 0.05 * rho
    rho_i = 0.95 * rho / iterations
  
    noisy_count = gaussian_mech_zCDP_vec([len(X_train)], 1, rho_count)[0]

    for i in range(iterations):
        clipped_gradient_sum = gradient_sum(theta, X_train, y_train, b)
        noisy_gradient_sum = np.array(gaussian_mech_zCDP_vec(clipped_gradient_sum, b, rho_i))
        noisy_avg_gradient = noisy_gradient_sum / noisy_count
        theta = theta - noisy_avg_gradient

    return theta

## Work

Predictive regression questions to answer:

* Sex
* Age
* Successful 3rd attempt
* Tested vs. untested
* Federation
* Equipment

In [13]:
print(df.columns)

Index(['Name', 'Sex', 'Event', 'Equipment', 'Age', 'AgeClass',
       'BirthYearClass', 'Division', 'BodyweightKg', 'WeightClassKg',
       'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Best3SquatKg',
       'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Best3BenchKg',
       'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg',
       'Best3DeadliftKg', 'TotalKg', 'Place', 'Dots', 'Wilks', 'Glossbrenner',
       'Goodlift', 'Tested', 'Country', 'State', 'Federation',
       'ParentFederation', 'Date', 'MeetCountry', 'MeetState', 'MeetTown',
       'MeetName'],
      dtype='object')
